In [1]:
import os
from groq import Groq
from dotenv import load_dotenv
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import requests
import tiktoken

In [2]:
# load environment variables from .env file
load_dotenv()

True

### Getting the raw data from repository

In [3]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Index the raw data with elasticsearch

Exercices 1 and 2

In [4]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client = Elasticsearch("http://localhost:9200")

# getting hash version of elasticsearch client
print(f"Elasticsearch build hash is: {es_client.info()["version"]["build_hash"]}")

es_client.indices.create(index=index_name, body=index_settings)

# we use the `index` method from the Elasticsearch client to index each document
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

Elasticsearch build hash is: dbcbbbd0bc4924cfeb28929dc05d82d662c527b7


  0%|          | 0/948 [00:00<?, ?it/s]

### Execute query against the indexed data

Exercices 3 and 4

In [5]:
def search_elasticsearch(index_name, query, fields, search_type, result_size=5, course=None):
    search_query = {
        "size": result_size,
        "query": {
            "bool": {
                "must": [
                    {
                        "multi_match": {
                            "query": query,
                            "fields": fields,
                            "type": search_type
                        }
                    }
                ],
                # Only add filter if course is provided
                **(
                    {"filter": [{"term": {"course": course}}]}
                    if course else {}
                )
            }
        }
    }  
    
    response = es_client.search(index=index_name, body=search_query)
    
    return response

In [6]:
query_k8s = "How do execute a command on a Kubernetes pod?"

response_k8s = search_elasticsearch(
    index_name=index_name,
    query=query_k8s,
    fields=["question^4", "text"],
    search_type="best_fields",
)

# print score of the first hit of the the first question aboud kubernetes
print(f"Score of kubernetes first hit is: {response_k8s["hits"]["hits"][0]["_score"]}")

query_docker =  "How do copy a file to a Docker container?"

response_docker = search_elasticsearch(
    index_name=index_name,
    query=query_docker,
    fields=["question^4", "text"],
    search_type="best_fields",
    course="machine-learning-zoomcamp",
    result_size=3
)

# print the third hit of the question about docker
print(f"Third hit of the question about docker is: {response_docker['hits']['hits'][2]['_source']["question"]}")

Score of kubernetes first hit is: 42.910965
Third hit of the question about docker is: How do I copy files from a different folder into docker container’s working directory?


### Building a prompt

In [7]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
    """.strip()

    context_template = """
Q: {question}
A: {text}
    """.strip()
    
    context = ""
    
    for doc in search_results:
        context = context + context_template.format(question=doc["question"], text=doc["text"]) +"\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [8]:
response_docker_docs = []

for hit in response_docker['hits']['hits']:
    response_docker_docs.append(hit['_source'])

prompt_docker = build_prompt(query_docker, response_docker_docs)

# print the length of the prompt
print(f"Length of the prompt is: {len(prompt_docker)} characters")

Length of the prompt is: 1446 characters


### Tokens in prompt

In [9]:
encoding = tiktoken.encoding_for_model("gpt-4o")

# calculate the number of tokens in the prompt
print(f"Number of tokens in the prompt is: {len(encoding.encode(prompt_docker))} tokens")

Number of tokens in the prompt is: 320 tokens


### Send the prompt to the LLM - Groq - Bonus

In [10]:
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

def llm(prompt):
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

groq_response = llm(prompt_docker)

In [11]:
print(groq_response)

To copy a file to a Docker container, you can use the `docker cp` command. The basic syntax is as follows: 
`docker cp /path/to/local/file_or_directory container_id:/path/in/container` 

You need to replace `/path/to/local/file_or_directory` with the path of the file on your local machine, `container_id` with the ID of the Docker container, and `/path/in/container` with the path where you want to copy the file in the container.


### Calculate the cost of requests - bonus

In [12]:
sent_token = 150
received_token = 250

cost_input_per_token = 2 / 1000000
cost_per_output_token = 8 / 1000000

cost_per_1000_request = ((sent_token * cost_input_per_token) + (received_token * cost_per_output_token)) * 1000

print(f"Cost per 1000 requests given an average of {sent_token} sent tokens and an average of {received_token} received token per request, is: ${cost_per_1000_request:.6f}")

Cost per 1000 requests given an average of 150 sent tokens and an average of 250 received token per request, is: $2.300000
